In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import linear_kernel

In [2]:
## rerun without normalisation

In [3]:
anime = pd.read_csv('anime.csv')
ratings = pd.read_csv('rating.csv')

In [4]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [5]:
# anime.info()
# anime['episodes'].unique()
anime['episodes'].replace('Unknown',0, inplace=True)

In [6]:
anime[anime['name']=='Tenkuu Danzai Skelter+Heaven']

,anime_id,name,genre,type,episodes,rating,members
8058,3287,Tenkuu Danzai Skelter+Heaven,"Mecha, Sci-Fi",OVA,1,2.0,7680


In [7]:
anime_titles = anime['name']
anime_titles

0                                           Kimi no Na wa.
1                         Fullmetal Alchemist: Brotherhood
2                                                 Gintama°
3                                              Steins;Gate
4                                            Gintama&#039;
                               ...                        
12289         Toushindai My Lover: Minami tai Mecha-Minami
12290                                          Under World
12291                       Violence Gekiga David no Hoshi
12292    Violence Gekiga Shin David no Hoshi: Inma Dens...
12293                     Yasuji no Pornorama: Yacchimae!!
Name: name, Length: 12294, dtype: object

In [8]:
ratings.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [9]:
ratings['rating'].unique()

array([-1, 10,  8,  6,  9,  7,  3,  5,  4,  1,  2], dtype=int64)

In [10]:
anime['rating'].replace(-1,1, inplace=True)
ratings['rating'].replace(-1,1, inplace=True)

In [11]:
# ratings.info()
ratings.shape[0]

7813737

In [12]:
# creating dummies for genres
genre_pd = anime['genre'].str.split(', ').str.join('|').str.get_dummies()
genre_pd.head()

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
genre_pd = genre_pd.astype('int32')

In [14]:
type_pd = anime['type'].str.get_dummies()
type_pd.head()

,Movie,Music,ONA,OVA,Special,TV
0,1,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [15]:
type_pd=type_pd.astype('int32')

In [16]:
anime_df = pd.concat([anime[['anime_id','rating','members']],genre_pd,type_pd],axis=1)
anime_df.head()

,anime_id,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,...,Thriller,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV
0,32281,9.37,200630,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,5114,9.26,793665,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,28977,9.25,114262,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9253,9.17,673572,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,9969,9.16,151266,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
anime_df[['anime_id','members']] = anime_df[['anime_id','members']].astype('int32')
anime_df['rating'] = anime_df['rating'].astype('float32')

In [18]:
len(anime_df['anime_id'].unique()) == anime_df.shape[0]

True

In [19]:
# !pip install -q scikit-surprise
from surprise import Dataset, Reader

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [21]:
# normalise ratings and number of members (popularity)
# anime_df['rating']=(anime_df['rating']-anime_df['rating'].mean())/anime_df['rating'].std()
anime_df['members']=(anime_df['members']-anime_df['members'].mean())/anime_df['members'].std()

In [22]:
cat_df=anime_df.drop('anime_id', axis=1)
cat_df.head()

,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,...,Thriller,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV
0,9.37,3.330106,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,9.26,14.147831,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,9.25,1.754642,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9.17,11.957179,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,9.16,2.429643,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [23]:
cat_df.fillna(0, inplace=True)

In [24]:
similarity_matrix = linear_kernel(cat_df,cat_df)
similarity_matrix

array([[103.88650334, 134.87997626,  92.5156426 , ...,  44.6411524 ,
         45.5754787 ,  51.07107422],
       [134.87997626, 293.90872041, 113.47937996, ...,  40.58156484,
         41.49620869,  45.93249251],
       [ 92.5156426 , 113.47937996,  96.64126848, ...,  44.56860236,
         45.49219318,  49.93113712],
       ...,
       [ 44.6411524 ,  40.58156484,  44.56860236, ...,  25.92044887,
         26.40870978,  27.75130596],
       [ 45.5754787 ,  41.49620869,  45.49219318, ...,  26.40870978,
         26.90697133,  28.29756794],
       [ 51.07107422,  45.93249251,  49.93113712, ...,  27.75130596,
         28.29756794,  31.91856494]])

In [52]:
def similarity_matrix(cat_df):
    from sklearn.metrics.pairwise import linear_kernel
    return linear_kernel(cat_df, cat_df)

In [41]:
# mapping = pd.Series(anime_df.index, index=anime['anime_id'])
# mapping

anime_id
32281        0
5114         1
28977        2
9253         3
9969         4
         ...  
9316     12289
5543     12290
5621     12291
6133     12292
26081    12293
Length: 12294, dtype: int64

In [44]:
def mapping():
    return pd.Series(anime_df.index, index=anime['anime_id'])

In [53]:
def recommend_on_genre(anime_movies, num_recommended: int = 15):
    anime_index = mapping()[anime_movies]
    similarity = list(enumerate(similarity_matrix(cat_df)[anime_index]))
    similarity = sorted(similarity, key=lambda x: x[1], reverse=True)
    similarity = similarity[1:num_recommended]
    anime_indices = [i[0] for i in similarity]
    return anime['anime_id'].iloc[anime_indices].reset_index()['anime_id'].tolist()

In [45]:
def recommend_on_genre(anime_movies, num_recommended: int = 15):
    anime_index = mapping(anime_movies)
    similarity = list(enumerate(similarity_matrix[anime_index]))
    similarity = sorted(similarity, key=lambda x: x[1], reverse=True)
    similarity = similarity[1:num_recommended]
    anime_indices = [i[0] for i in similarity]
    return anime['anime_id'].iloc[anime_indices].reset_index()['anime_id'].tolist()

In [54]:
a = recommend_on_genre(28977)
a

[1535,
 16498,
 9253,
 1575,
 2904,
 30276,
 6547,
 2001,
 11757,
 918,
 1,
 11061,
 4224,
 121]

In [55]:
def get_rating(movie_id, df=anime_df):
    return df['rating'][df['anime_id']==movie_id].iloc[0]

In [56]:
get_rating(11757)

7.83

In [57]:
def user_top_anime(user_id, df=ratings, top_n: int =10, rating_min=5):
    """
    args: user id, optional: ratings dataframe if other than default, number of movies to return, minimum rating
    returns: dataframe with n top movie id's for a specified user
    """
    recommended_df = df[['anime_id','rating']].loc[df['user_id']==user_id]
    return recommended_df[recommended_df['rating']>=rating_min].nlargest(top_n, 'rating')

In [59]:
user_top_anime(3)

,anime_id,rating
159,199,10
165,813,10
173,1535,10
183,5114,10
195,9989,10
203,11771,10
208,14513,10
209,16498,10
212,16894,10
216,18115,10


In [60]:
def seen(movie, user, df=ratings):
    """function checks if user has seen and reviewed recommended movie
    args: movie id, user id, optional: ratings dataframe if other than default
    returns True/False for each movie
    """
    isin = movie in df['anime_id'].loc[df['user_id']==user]
    return isin

In [64]:
def combine(user, df1=anime_df, df2=ratings, n=5):
    """
    main function
    run user_top_anime
    apply recommend_on_genre to each top title
    group count recommended titles
    apply get_rating 
    apply seen
    sort by rating & count recommended where seen = False
    return top n
    """
    from itertools import chain
    from collections import Counter
    # generate top 10 animes for a user
    top_anime = user_top_anime(user, df2)
    recommended_list = []
    #for each anime from user's top list recommend similar animes
    for anime, _ in top_anime.iterrows():
        recommended_list.append(recommend_on_genre(anime))
    final_list = chain(*recommended_list)
    # add number of times they appear in recommendations
    rec_count = Counter(final_list)
    df = pd.DataFrame.from_dict(rec_count, orient='index').reset_index()
    print(df.columns)
    df['rating']=df['index'].apply(get_rating)
    df['seen']=df['index'].apply(lambda x: seen(x, user=user))
    df.rename(columns={'index':'anime_id',0:'num_recommendations'}, inplace=True)
    return df[df['seen']==False].sort_values(by=['num_recommendations','rating'],ascending=False).head(n)

In [62]:
# ratings[ratings['user_id']==1].sort_values(by='rating', )

In [65]:
combine(5)

Index(['index', 0], dtype='object')


,anime_id,num_recommendations,rating,seen
1,30120,9,9.60,False
9,23005,9,9.50,False
6,28977,9,9.25,False
10,9969,9,9.16,False
2,32281,8,9.37,False
